### Trees

While all tree are graphs, `not` all graphs are tree.  
A graph is a tree when it has `no` cycles.  
In a graphs all nodes must be `connected`. 

### Jargon

Each node is called a `vertex`.  
The line between the nodes (vertices) are called `edges`.  

### Graphs

A graphs is a data structure specialized in `relationships`.  
For example, in a social network, each person is represented by a `node`.  
Each `line` represents a friendship with another person.  
We can look up for friends in just `O(1)` steps.

In [2]:
friends = {
    'Alice': ['Bob', 'Diana', 'Fred'],
    'Bob': ['Alice', 'Cynthia', 'Diana'],
    'Cynthia': ['Bob'],
    'Diana': ['Alice', 'Bob', 'Fred'],
    'Elise': ['Fred'],
    'Fred': ['Alice', 'Diana', 'Elise'],
}

print(friends['Alice'])

['Bob', 'Diana', 'Fred']
